In [9]:
class Card:
  def __init__(self, colour, number):
    assert isinstance(number, int)
    self.colour = colour
    self.number = number

  def __repr__(self):
    return f'{self.colour} {self.number}'

from itertools import combinations
class CollectionOfCards:
    def __init__(self, collection):
        self.collection = []
        for card in collection:
          # Pastikan card adalah objek Card
          if isinstance(card, Card):
            self.collection.append(card)
          else:
            colour,number = card.split(' ')
            self.collection.append(Card(colour, int(number)))

    def __str__(self):
        return f"{self.collection}"

    def __repr__(self):
        return f"{self.collection}"

    def is_valid_group(self,subset=None):
          if subset is None:
            subset = self.collection
          # Check whether there are at least 3 cards in the collection
          if len(subset) < 3:
              return False

          # Finding same colour but consecutive numbers (valid1)
          valid1 = False
          sorted_cards = sorted(subset, key=lambda card: (card.colour, int(card.number)))
          counter1 = 1

          for i in range(1,len(sorted_cards)):
            if sorted_cards[i].colour == sorted_cards[i-1].colour:
              if sorted_cards[i].number == sorted_cards[i-1].number + 1:
                counter1 += 1
              else:
                counter1 = 1
            else:
              counter1 = 1
              break
          if counter1 >= 3:
            valid1 = True
         
          # Finding same number but different colours (valid2)
          valid2 = False
          counter2 = 0
          group_colours = []
          for card in subset:
            if card.colour not in group_colours and card.number == subset[0].number:
              group_colours.append(card.colour)
              counter2 += 1
            else:
              counter2 = 1
          if counter2 >= 3:
            valid2 = True

          return valid1 or valid2


    def find_valid_group(self):

      for group_size in range(len(self.collection), 2, -1):  # Groups must be at least 3 cards
        for subset in combinations(self.collection, group_size):
          if self.is_valid_group(subset):
            return list(subset)
      return None


    def find_largest_valid_group(self):
        valid_groups = []
        for group_size in range(len(self.collection), 2, -1):  # Groups must be at least 3 cards
          for subset in combinations(self.collection, group_size):
            if self.is_valid_group(subset):
              valid_groups.append(subset)
        largest_size = 0
        largest_group = None
        for a in valid_groups:
          if len(a) > largest_size:
            largest_group = a
            largest_size = len(a)


        return largest_group



In [11]:
CollectionOfCards(['blue 3', 'blue 4', 'red 4', 'green 4']).find_valid_group()

[blue 4, red 4, green 4]

In [1]:
import pygame
import random
import os
from dataclasses import dataclass
from typing import List, Tuple

class Card:
  color: str
  number: int
  image: pygame.Surface = None
  rect: pygame.Rect = None
    
  def __init__(self, color, number):
    assert isinstance(number, int)
    self.color = color
    self.number = number

  def __repr__(self):
    return f'{self.colour} {self.number}'


class CardGame:
    def __init__(self):
        self.colors = ['red', 'blue', 'yellow', 'green']
        self.numbers = range(1, 11)  # 1 to 10
        self.deck = []
        self.players_hands = []
        self.card_images = {}  # Dictionary to store loaded images
        self.load_card_images()
        
    def load_card_images(self):
        """
        Load card images from a directory.
        Expected image naming format: 'color_number.png'
        Example: 'red_1.png', 'blue_10.png'
        """
        # Specify the directory where your card images are stored
        image_dir = 'cards'  # Create a 'cards' folder in your project directory
        
        # Make sure the images directory exists
        if not os.path.exists(image_dir):
            os.makedirs(image_dir)
            print(f"Created '{image_dir}' directory. Please place your card images there.")
            return
        
        # Load each card image
        for color in self.colors:
            for number in self.numbers:
                # Construct the filename
                filename = f"{color}_{number}.png"
                filepath = os.path.join(image_dir, filename)
                
                try:
                    # Load and scale the image (adjust size as needed)
                    original_image = pygame.image.load(filepath)
                    scaled_image = pygame.transform.scale(original_image, (80, 120))
                    self.card_images[f"{color}_{number}"] = scaled_image
                except pygame.error:
                    print(f"Couldn't load image: {filepath}")
                    # Create a placeholder if image is missing
                    placeholder = pygame.Surface((80, 120))
                    placeholder.fill((200, 200, 200))  # Gray background
                    font = pygame.font.Font(None, 36)
                    text = font.render(f"{color[0]}{number}", True, (0, 0, 0))
                    text_rect = text.get_rect(center=(40, 60))
                    placeholder.blit(text, text_rect)
                    self.card_images[f"{color}_{number}"] = placeholder
        
    def create_deck(self):
        # Create two of each card
        for _ in range(2):
            for color in self.colors:
                for number in self.numbers:
                    card = Card(color, number)
                    # Assign the corresponding image
                    card.image = self.card_images[f"{color}_{number}"]
                    self.deck.append(card)
        
        # Shuffle the deck
        random.shuffle(self.deck)
    
    def deal_initial_hands(self, num_players):
        if num_players not in [2, 3]:
            raise ValueError("Game supports only 2 or 3 players")
        
        self.players_hands = [[] for _ in range(num_players)]
        
        # Deal 5 cards to each player
        for _ in range(5):
            for player in range(num_players):
                if self.deck:  # Check if deck is not empty
                    card = self.deck.pop()
                    self.players_hands[player].append(card)
    
    def draw_cards(self, screen, card_width=80, card_height=120):
        # Draw cards for each player
        for player_idx, hand in enumerate(self.players_hands):
            for card_idx, card in enumerate(hand):
                # Position cards for each player
                if player_idx == 0:  # Bottom player
                    x = 100 + card_idx * (card_width + 10)
                    y = screen.get_height() - card_height - 20
                elif player_idx == 1:  # Top player
                    x = 100 + card_idx * (card_width + 10)
                    y = 20
                else:  # Third player (if exists) on right side
                    x = screen.get_width() - card_width - 20
                    y = 100 + card_idx * (card_height + 10)
                
                # Create rectangle for card position
                card.rect = pygame.Rect(x, y, card_width, card_height)
                
                # Draw the card image
                screen.blit(card.image, card.rect)
                

def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    clock = pygame.time.Clock()
    
    game = CardGame()
    game.create_deck()
    game.deal_initial_hands(2)  # Change to 3 for three players
    
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False
        
        screen.fill((0, 100, 0))  # Green table background
        game.draw_cards(screen)
        pygame.display.flip()
        clock.tick(60)
    
    pygame.quit()

if __name__ == "__main__":
    main()

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [1]:
import pygame
import random
import os
from dataclasses import dataclass
from typing import List, Tuple

class Card:
    color: str
    number: int
    image: pygame.Surface = None
    rect: pygame.Rect = None

    def __init__(self, color, number):
        assert isinstance(number, int)
        self.color = color
        self.number = number

    def __repr__(self):
        return f'{self.color} {self.number}'


class CardGame:
    def __init__(self):
        self.colors = ['red', 'blue', 'yellow', 'green']
        self.numbers = range(1, 11)  # 1 to 10
        self.deck = []
        self.players_hands = []
        self.card_images = {}  # Dictionary to store loaded images
        self.load_card_images()
        
    def load_card_images(self):
        image_dir = 'cards'
        
        if not os.path.exists(image_dir):
            os.makedirs(image_dir)
            print(f"Created '{image_dir}' directory. Please place your card images there.")
            return
        
        for color in self.colors:
            for number in self.numbers:
                filename = f"{color}_{number}.png"
                filepath = os.path.join(image_dir, filename)
                
                try:
                    original_image = pygame.image.load(filepath)
                    scaled_image = pygame.transform.scale(original_image, (80, 120))
                    self.card_images[f"{color}_{number}"] = scaled_image
                except pygame.error:
                    print(f"Couldn't load image: {filepath}")
                    placeholder = pygame.Surface((80, 120))
                    placeholder.fill((200, 200, 200))
                    font = pygame.font.Font(None, 36)
                    text = font.render(f"{color[0]}{number}", True, (0, 0, 0))
                    text_rect = text.get_rect(center=(40, 60))
                    placeholder.blit(text, text_rect)
                    self.card_images[f"{color}_{number}"] = placeholder
        
    def create_deck(self):
        for _ in range(2):
            for color in self.colors:
                for number in self.numbers:
                    card = Card(color, number)
                    card.image = self.card_images[f"{color}_{number}"]
                    self.deck.append(card)
        
        random.shuffle(self.deck)
    
    def deal_initial_hands(self, num_players):
        if num_players not in [2, 3]:
            raise ValueError("Game supports only 2 or 3 players")
        
        self.players_hands = [[] for _ in range(num_players)]
        
        for _ in range(5):
            for player in range(num_players):
                if self.deck:
                    card = self.deck.pop()
                    self.players_hands[player].append(card)
    
    def draw_cards(self, screen, card_width=80, card_height=120):
        for player_idx, hand in enumerate(self.players_hands):
            for card_idx, card in enumerate(hand):
                if player_idx == 0:
                    x = 100 + card_idx * (card_width + 10)
                    y = screen.get_height() - card_height - 20
                elif player_idx == 1:
                    x = 100 + card_idx * (card_width + 10)
                    y = 20
                else:
                    x = screen.get_width() - card_width - 20
                    y = 100 + card_idx * (card_height + 10)
                
                card.rect = pygame.Rect(x, y, card_width, card_height)
                screen.blit(card.image, card.rect)

def show_home_screen(screen):
    font_title = pygame.font.SysFont('Comic Sans MS', 48)
    font_button = pygame.font.SysFont('Courier', 36)
    
    # Draw background and text
    screen.fill((39, 39, 39))  # Green background
    
    title_text = font_title.render("Notty Game by Funt4stic Team", True, (255, 255, 255))
    title_rect = title_text.get_rect(center=(screen.get_width() // 2, 100))
    screen.blit(title_text, title_rect)

    # Play button
    play_button = font_button.render("Play", True, (255, 255, 255))
    play_button_rect = play_button.get_rect(center=(screen.get_width() // 2, 200))
    
    # Quit button
    quit_button = font_button.render("Quit", True, (255, 255, 255))
    quit_button_rect = quit_button.get_rect(center=(screen.get_width() // 2, 300))
    
    screen.blit(play_button, play_button_rect)
    screen.blit(quit_button, quit_button_rect)
    
    pygame.display.flip()
    
    return play_button_rect, quit_button_rect

def show_player_selection(screen):
    font_button = pygame.font.SysFont('Comis Sans MS', 36)

    screen.fill((39, 39, 39))  # Silver background
    
    select_text = pygame.font.SysFont('Courier', 48).render("Select number of players", True, (255, 255, 255))
    select_rect = select_text.get_rect(center=(screen.get_width() // 2, 100))
    screen.blit(select_text, select_rect)

    # 2 Players button
    two_players_button = font_button.render("2 Players", True, (255, 255, 255))
    two_players_button_rect = two_players_button.get_rect(center=(screen.get_width() // 2, 200))
    
    # 3 Players button
    three_players_button = font_button.render("3 Players", True, (255, 255, 255))
    three_players_button_rect = three_players_button.get_rect(center=(screen.get_width() // 2, 300))
    
    screen.blit(two_players_button, two_players_button_rect)
    screen.blit(three_players_button, three_players_button_rect)
    
    pygame.display.flip()
    
    return two_players_button_rect, three_players_button_rect

def handle_menu_selection(event, play_button_rect, quit_button_rect, two_players_button_rect, three_players_button_rect):
    if event.type == pygame.MOUSEBUTTONDOWN:
        mouse_pos = pygame.mouse.get_pos()
        if play_button_rect and play_button_rect.collidepoint(mouse_pos):
            return 'play'
        elif quit_button_rect and quit_button_rect.collidepoint(mouse_pos):
            return 'quit'
        elif two_players_button_rect and two_players_button_rect.collidepoint(mouse_pos):
            return '2_players'
        elif three_players_button_rect and three_players_button_rect.collidepoint(mouse_pos):
            return '3_players'
    return None

def main():
    pygame.init()
    screen = pygame.display.set_mode((800, 600))
    clock = pygame.time.Clock()
    
    game = CardGame()
    
    # Main menu loop
    menu_running = True
    while menu_running:
        play_button_rect, quit_button_rect = show_home_screen(screen)
        
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                menu_running = False
            selection = handle_menu_selection(event, play_button_rect, quit_button_rect, None, None)
            if selection == 'quit':
                menu_running = False
            elif selection == 'play':
                # Show player selection screen
                two_players_button_rect, three_players_button_rect = show_player_selection(screen)
                
                # Handle player selection
                waiting = True
                while waiting:
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            waiting = False
                            menu_running = False
                        selection = handle_menu_selection(event, None, None, two_players_button_rect, three_players_button_rect)
                        if selection == '2_players':
                            num_players = 2
                            waiting = False
                        elif selection == '3_players':
                            num_players = 3
                            waiting = False
                
                # Start the game
                game.create_deck()
                game.deal_initial_hands(num_players)
                
                # Game loop
                game_running = True
                while game_running:
                    for event in pygame.event.get():
                        if event.type == pygame.QUIT:
                            game_running = False
                    
                    screen.fill((39, 39, 39))
                    game.draw_cards(screen)
                    pygame.display.flip()
                    clock.tick(60)
                
                # Return to main menu after game ends
                menu_running = True
    
    pygame.quit()

if __name__ == "__main__":
    main()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
